In [1]:
import os

from konlpy.tag import Komoran, Kkma, Hannanum, Okt, Mecab
from pororo import Pororo

class DataGenerator:
  def __init__(self):
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
    self.qg = Pororo(task="qg", lang="ko")
    self.ner = Pororo(task="ner", lang="ko")
    self.mecab = Mecab()
    self.okt = Okt()
    self.komoran = Komoran()
    self.kkma = Kkma()
    self.han = Hannanum()
    self.id = 1

  def delete_duplicate(self, answer_list): 
    new_answer_list = []
    duplicate_list = []
    for answer in answer_list:
      if answer not in new_answer_list:
        new_answer_list.append(answer)
      else:
        duplicate_list.append(answer)
    duplicate_list = list(set(duplicate_list))
    for dup_ans in duplicate_list:
      new_answer_list.remove(dup_ans)
    return new_answer_list

  def get_only_komoran_NNP(self, answer_list):
    new_answer_list = []
    for answer in answer_list:
      if len(self.han.pos(answer)) > 1 and len(self.mecab.pos(answer)) > 1 and len(self.kkma.pos(answer)) > 1 and len(self.okt.pos(answer)) > 1:
        # 여기 정규표현식으로 0월 0일 만 걸러주게 만들기
        ner_data = self.ner(answer)
        if len(ner_data) == 1 and ner_data[0][1] == 'DATE':
          continue
        new_answer_list.append(answer)
    return new_answer_list

  def get_komoran_NNP(self, context):
    pos_list = self.komoran.pos(context)
    pos_list = list(filter(lambda x: x[1] == 'NNP', pos_list))
    answer_list = [ answer[0] for answer in pos_list ]
    answer_list = self.delete_duplicate(answer_list)
    answer_list = self.get_only_komoran_NNP(answer_list)
    return answer_list
  
  def get_date(self, context):
    ners = self.ner(context)
    quentity_date_ner = list(filter(lambda x: x[1] == 'DATE', ners))
    return [answer[0] for answer in quentity_date_ner]
  
  def get_date_question(self, context):
    answers = self.get_date(context)
    answers = self.delete_duplicate(answers)
    return answers
  
  def get_question_N(self, context):
    answer_list = self.get_komoran_NNP(context)
    if len(answer_list) == 0:
      return [], []
    return answer_list, self.qg(answer_list, context)
  
  def get_question_Q(self, context):
    answer_list = self.get_date_question(context)
    if len(answer_list) == 0:
      return [], []
    return answer_list, self.qg(answer_list, context)
    
  def generate_question_with_dataset_format(self, wiki_data, q_type = "N"):
    datasets = []
    if q_type == "N":
      answer_list, question_list = self.get_question_N(wiki_data['context'])
    elif q_type == "Q":
      answer_list, question_list = self.get_question_Q(wiki_data['context'])
    else:
      raise "정확한 q_type을 입력하세요. N or Q"
    for answer, question in zip(answer_list, question_list):
      data = {
        'context': wiki_data['context'],
        'question': question,
        'id': self.id,
        'title': wiki_data['title'],
        'document_id': wiki_data['document_id'],
        'answers': {
          'answer_start': [wiki_data['context'].index(answer)],
          'text': [answer]
        }
      }
      self.id += 1
      datasets.append(data)
    return datasets
    

In [2]:
from datasets import load_from_disk

wiki_datasets = load_from_disk("/opt/ml/data/wiki_datasets")

In [3]:
dg = DataGenerator()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [4]:
from tqdm import tqdm

datasets = {
  'context': [],
  'question': [],
  'id': [],
  'title': [],
  'document_id': [],
  'answers': []
}
error_data = []
for index in tqdm(range(0, 5)):
  data_list = []
  try:
    data_list.extend(dg.generate_question_with_dataset_format(wiki_datasets[index]))
  except:
    error_data.append(index)
  # data_list.extend(dg.generate_question_with_dataset_format(wiki_datasets[index], q_type='Q'))
  for data in data_list:
    for key in datasets.keys():
      datasets[key].append(data[key])

100%|██████████| 5/5 [04:12<00:00, 50.46s/it]


In [5]:
datasets['answers']

[{'answer_start': [6], 'text': ['나라 목록']},
 {'answer_start': [103], 'text': ['바티칸 시국']},
 {'answer_start': [128], 'text': ['국제 기구']},
 {'answer_start': [206], 'text': ['데 팍토']},
 {'answer_start': [520], 'text': ['바티칸 시국']},
 {'answer_start': [610], 'text': ['유엔 회원국']},
 {'answer_start': [642], 'text': ['니우에']},
 {'answer_start': [662], 'text': ['사하라 아랍 민주 공화국']},
 {'answer_start': [694], 'text': ['쿡 제도']},
 {'answer_start': [1295], 'text': ['독일 제국']},
 {'answer_start': [1336], 'text': ['우크라이나 소비에트 사회주의 공화국']},
 {'answer_start': [1369], 'text': ['벨로루시 소비에트 사회주의 공화국']},
 {'answer_start': [248], 'text': ['한국 전쟁']},
 {'answer_start': [278], 'text': ['그 후']},
 {'answer_start': [358], 'text': ['모로이 사부로']},
 {'answer_start': [409], 'text': ['아르놀트 쇤베르크']},
 {'answer_start': [465], 'text': ['쾰른 대학교']},
 {'answer_start': [682], 'text': ['세계의 역사']},
 {'answer_start': [807], 'text': ['앨런 카']},
 {'answer_start': [1436], 'text': ['노버트 위너']},
 {'answer_start': [2118], 'text': ['사이에서']},
 {'answer_sta

In [9]:
from datasets import Dataset
new_train_data = Dataset.from_dict(datasets)

In [31]:
new_train_data[40]

{'context': '저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의 서적 대부분이 존재한다. 외국 번역작품이나 저자가 무료보기를 인정한 현대작품도 포함된다. 장르는 정치부터 취미까지 다양하지만, 비교적 문학작품(시대소설, 추리소설등의 오락작품 포함)이 많다. 유명작가의 작품이 모두 갖춰져있진 않지만 그래도 일본어작품에 관련해서는 충실하게 갖춰진 편이다. (번역작품의 경우 번역저작권을 문제로 수가 많지 않다.)\n\n잘 알려지지 않은 작품을 보존, 소개하는 장점도 있다. 작품 텍스트화는 지금도 현재진행형이며 2011년 3월 15일 현재 등록작품수가 1만권이 넘었다.\n고전작가인 모리 오가이, 나츠메 소세키, 아쿠타가와 류노스케, 최근의 작가로는 나카지마 아츠시, 다자이 오사무, 하야시 후미코, 미야모토 유리코, 호리 다쓰오, 사카구치 안고, 다카무라 고타로, 나가이 가후, 요시카와 에이지 등 인물의 작품이 있다.',
 'document_id': 4,
 'answers': {'answer_start': [416], 'text': ['사카구치 안고']},
 'id': 41,
 'question': '호리 다쓰오의 저자가 누구야?',
 'title': '아오조라 문고'}

In [27]:
from tqdm import tqdm

datasets2 = {
  'context': [],
  'question': [],
  'id': [],
  'title': [],
  'document_id': [],
  'answers': []
}
error_data = []
for index in tqdm(range(0, 5)):
  data_list = []
  # try:
  #   data_list.extend(dg.generate_question_with_dataset_format(wiki_datasets[index]))
  # except:
  #   error_data.append(index)
  try:
    data_list.extend(dg.generate_question_with_dataset_format(wiki_datasets[index], q_type='Q'))
  except:
    error_data.append(index)
  for data in data_list:
    for key in datasets.keys():
      datasets2[key].append(data[key])

100%|██████████| 5/5 [05:07<00:00, 61.54s/it]


In [33]:
datasets2['answers']

[{'answer_start': [16], 'text': ['1933년']},
 {'answer_start': [19], 'text': ['일제 강점기']},
 {'answer_start': [221], 'text': ['1949년']},
 {'answer_start': [291], 'text': ['1952년']},
 {'answer_start': [315], 'text': ['2년 후']},
 {'answer_start': [427], 'text': ['1956년']},
 {'answer_start': [512], 'text': ['1년 후']},
 {'answer_start': [608], 'text': ['1958년']},
 {'answer_start': [845], 'text': ['1959년']},
 {'answer_start': [1170], 'text': ['1960년대 초반']},
 {'answer_start': [1535], 'text': ['2년여 동안']},
 {'answer_start': [1862], 'text': ['20세기']},
 {'answer_start': [2090], 'text': ['2015-5-9']},
 {'answer_start': [2172], 'text': ['1964년']},
 {'answer_start': [2305], 'text': ['1965년']},
 {'answer_start': [2564], 'text': ['1967년']},
 {'answer_start': [2857], 'text': ['1974년부터']},
 {'answer_start': [3123], 'text': ['1960년대 후반부터']},
 {'answer_start': [2857], 'text': ['1974년']},
 {'answer_start': [3271], 'text': ['1959-1973']},
 {'answer_start': [3334], 'text': ['1982년']},
 {'answer_start': [3415], '

In [32]:
from datasets import Dataset
new_train_data2 = Dataset.from_dict(datasets2)

In [34]:
new_train_data2[10]

{'context': '현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친일파인 아버지 백낙승과 어머니 조종희 사이의 3남 2녀 중 막내로 태어났다. 그후 종로구 창신동 197번지 소위 "큰대문집"에서 18세까지 살았다. 수송국민학교와 경기제1고등보통학교를 다니면서 피아니스트 신재덕에게 피아노 연주를, 작이건우]]에게 작곡을 각각 배웠다. 이때 한국이 낳은 작곡가 김순남을 사사했다. 1949년 그는 홍콩 로이덴 스쿨로 전학했으며, 한국 전쟁이 발발하기 이전 가족이 일본으로 이주했다. 그 후 일본으로 건너가 1952년 도쿄 대학교 문과부에 입학했다. 2년 후 미술사학 및 미학으로 전공을 정했지만, 실제로는 일본 당대의 작곡가 모로이 사부로, 미학자 노무라 요시오 등에게서 작곡과, 음악사학을 공부했다. 졸업 논문은 ‘아르놀트 쇤베르크 연구’이다.\n\n1956년 백남준은 졸업과 함께 독일로 유학을 떠나 뮌헨 대학교 및 쾰른 대학교 등에서 서양의 건축, 음악사, 철학 등을 공부하였다. 뮌헨 대학교 입학 1년 후에는 프라이부르크 국립 음악 대학교로 옮겨 볼프강 포르트너 교수에게 배우지만, 곧 쇤베르크 이후 현대음악의 실험이 활발히 진행되던 다름슈타트 하기 강좌에 참여했다. 1958년 그 곳에서 현대음악가 존 케이지를 만나 그의 음악에 대한 파괴적 접근과 자유정신으로부터 깊은 영감을 얻었다. 이 영감은 "세계의 역사는 우리에게 알려준다. 주어진 게임에서 이길 수 없다면 규칙을 바꿔라" 라는 것으로 규정된다. 이후 1950년대부터 활발해지기 시작한 독일 라인 지역의 액션뮤직의 현장에서 백남준은 ‘아시아에서 온 문화테러리스트’(앨런 카프로)라고 불릴 정도의 탁월한 퍼포먼스 아티스트로 활약했다. 1959년 ‘존 케이지에게 보내는 경의’에서 음악적 콜라주와 함께 피아노를 부수는 퍼포먼스를 선보이는 것을 시작으로, 바이올린을 단숨에 파괴하거나(바이올린 솔로) 존 케이지가 착용한 넥타이를 잘라버리는 퍼포먼스(피아노 포르테를 위한 연습곡)가 특히 유